In [18]:
import configparser
import os
import duckdb

# Set MALLARD_CONFIG environment variable to where the config file is located
os.environ['MALLARD_CONFIG'] = '/data/mallard/config.ini'

# Get config file
config_file = os.getenv('MALLARD_CONFIG')
config = configparser.ConfigParser()
config.read(config_file)

db_file = config['DEFAULT']['db_file']

Here are all the tables in my database. tiingo_fundamentals_amended_distinct is in development, there is a function with description to create it, but it can be ignored for now.

In [19]:
with duckdb.connect(db_file) as con:
    con.sql("SHOW TABLES").show()

┌──────────────────────────────────────┐
│                 name                 │
│               varchar                │
├──────────────────────────────────────┤
│ alembic_version                      │
│ daily_metrics                        │
│ fundamental_metrics                  │
│ inflation                            │
│ tiingo_eod                           │
│ tiingo_fundamental_metrics           │
│ tiingo_fundamentals_amended          │
│ tiingo_fundamentals_amended_distinct │
│ tiingo_fundamentals_daily            │
│ tiingo_fundamentals_meta             │
│ tiingo_fundamentals_reported         │
│ tiingo_symbols                       │
├──────────────────────────────────────┤
│               12 rows                │
└──────────────────────────────────────┘


This has symbols taken from supported_tickers.csv, after filters have been applied. Note that Tiingo doesn't include the vendor_symbol_id (permaTicker) in this file.

In [20]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_symbols").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_symbols").show()
    con.sql("SELECT * FROM tiingo_symbols LIMIT 5").show()


┌────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name   │ column_type │  null   │   key   │ default │  extra  │
│    varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ symbol         │ VARCHAR     │ NO      │ PRI     │ NULL    │ NULL    │
│ exchange       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ asset_type     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ price_currency │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ start_date     │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ end_date       │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
└────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

┌───────────────┐
│ count(symbol) │
│     int64     │
├───────────────┤
│         10011 │
└───────────────┘

┌─────────┬──────────┬────────────┬────────────────┬────────────┬────────────┐
│ symbol

This has metadata for the fundamentals. Even if you don't have the fundamentals addon, we use this to ensure we're getting stocks that are **reporting** fundamentals to the SEC.

In [21]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_fundamentals_meta").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_fundamentals_meta").show()
    con.sql("SELECT * FROM tiingo_fundamentals_meta LIMIT 5").show()

┌────────────────────────┬──────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│      column_name       │       column_type        │  null   │   key   │ default │  extra  │
│        varchar         │         varchar          │ varchar │ varchar │ varchar │ varchar │
├────────────────────────┼──────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id       │ VARCHAR                  │ NO      │ PRI     │ NULL    │ NULL    │
│ symbol                 │ VARCHAR                  │ YES     │ NULL    │ NULL    │ NULL    │
│ name                   │ VARCHAR                  │ YES     │ NULL    │ NULL    │ NULL    │
│ is_active              │ BOOLEAN                  │ YES     │ NULL    │ NULL    │ NULL    │
│ is_adr                 │ BOOLEAN                  │ YES     │ NULL    │ NULL    │ NULL    │
│ sector                 │ VARCHAR                  │ YES     │ NULL    │ NULL    │ NULL    │
│ industry               │ VARCHAR                  │ YES   

There are several nuances to fundamentals data, see the docs: https://www.tiingo.com/documentation/fundamentals
You'll want to do a call to the descriptions endpoint to see what all the dataCodes mean.
You'll note I didn't normalize all the columns to make loading a little easier.

Fundamentals amended uses the fiscal period end date and has amendments (corrections) submitted to the SEC.

In [22]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_fundamentals_amended").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_fundamentals_amended").show()
    con.sql("SELECT * FROM tiingo_fundamentals_amended LIMIT 5").show()


┌──────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name    │ column_type │  null   │   key   │ default │  extra  │
│     varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ symbol           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date             │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ year             │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ quarter          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ statementType    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ dataCode         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ value            │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
└──────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘
┌───────────────┐
│ count

Fundamentals reported uses the filing date, which can be up to 90 days after the fiscal period end date. 
The data is as reported (no corrections) and suitable for backtesting against latest quarter or annual data.
Annual reports have quarter == 0.
Yes, there are ~10% fewer rows than amended and the earliest start date is 1993 vs. 1990. There is no filtering, that's straight from Tiingo. 

In [23]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_fundamentals_reported").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_fundamentals_reported").show()
    con.sql("SELECT * FROM tiingo_fundamentals_reported LIMIT 5").show()


┌──────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name    │ column_type │  null   │   key   │ default │  extra  │
│     varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ symbol           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date             │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ year             │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ quarter          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ statementType    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ dataCode         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ value            │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
└──────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘
┌───────────────┐
│ count

Fundamentals daily includes metrics generated by Tiingo, not Mallard. Unlike their other fundamental tables, it has a column per metric. 
The docs state they will add new metrics so this may change over time.

In [24]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE tiingo_fundamentals_daily").show()
    con.sql("SELECT COUNT(symbol) FROM tiingo_fundamentals_daily").show()
    con.sql("SELECT * FROM tiingo_fundamentals_daily LIMIT 5").show()


┌──────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name    │ column_type │  null   │   key   │ default │  extra  │
│     varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ symbol           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date             │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ market_cap       │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ enterprise_val   │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ pe_ratio         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ pb_ratio         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ trailing_peg_1y  │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
└──────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

┌───────────────┐
│ coun

Daily metrics are calculated by Mallard. It has a metric per column like fundamentals daily and they are created as needed per the config file.
Many metrics are planned, and as an OLAP DB, DuckDB has no problem with very wide tables. 
Note that MACD is a moving average so early values are NULL. 

In [25]:
with duckdb.connect(db_file) as con:
    con.sql("DESCRIBE daily_metrics").show()
    con.sql("SELECT COUNT(symbol) FROM daily_metrics").show()
    con.sql("SELECT * FROM daily_metrics LIMIT 5").show()


┌─────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│       column_name       │ column_type │  null   │   key   │ default │  extra  │
│         varchar         │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ vendor_symbol_id        │ VARCHAR     │ NO      │ PRI     │ NULL    │ NULL    │
│ symbol                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date                    │ DATE        │ NO      │ PRI     │ NULL    │ NULL    │
│ trading_value           │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ avg_daily_trading_value │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ macd                    │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ macd_signal             │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ macd_hist               │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ has_min_tradin